In [318]:
using Pkg
Pkg.activate(".")

  Activating project at `~/Desktop/Continuous MPS/MatrixProductBP.jl`


In [319]:
using Revise
using MatrixProductBP, MatrixProductBP.Models
using Graphs, IndexedGraphs, Statistics, Random, LinearAlgebra, TensorTrains, SparseArrays
import ProgressMeter; ProgressMeter.ijulia_behavior(:clear)
using TensorTrains: summary_compact
using SparseArrays
using JLD2
include("bp_fourier.jl");

In [320]:
seed = 4
rng = MersenneTwister(seed)

T = 10
# N = 3
# A = [0 1 1; 
#      1 0 0;
#      1 0 0;]
# N = 4
# A = [0 1 0 0; 
#      1 0 1 0; 
#      0 1 0 1; 
#      0 0 1 0]
# N = 7
# A = [0 1 0 0 0 0 0; 
#      1 0 1 0 0 0 0; 
#      0 1 0 1 0 0 0;
#      0 0 1 0 1 0 0;
#      0 0 0 1 0 1 0;
#      0 0 0 0 1 0 1;
#      0 0 0 0 0 1 0]
# N = 4
# A = [0 1 1 1;
#      1 0 0 1;
#      1 0 0 1;
#      1 1 1 0]
# !!! Here the calculations are wrong for edge 1->2 !!!
# N = 5
# A = [0 1 1 1 1; 
#      1 0 0 0 0; 
#      1 0 0 0 0; 
#      1 0 0 0 0; 
#      1 0 0 0 0]
# N = 6
# A = [0 1 1 1 0 0; 
#      1 0 0 0 1 0; 
#      1 0 0 0 0 1; 
#      1 0 0 0 0 0; 
#      0 1 0 0 0 0; 
#      0 0 1 0 0 0]
# N = 7
# A = [0 1 1 1 1 0 1;
#      1 0 0 0 1 0 0; 
#      1 0 0 0 0 1 0; 
#      1 0 0 0 0 0 0; 
#      1 1 0 0 0 0 0; 
#      0 0 1 0 0 0 0; 
#      1 0 0 0 0 0 0]
N = 8
A = [0 1 1 1 0 0 1 0;
     1 0 0 0 1 0 0 1; 
     1 0 0 0 0 1 0 0; 
     1 0 0 0 0 0 0 0; 
     0 1 0 0 0 0 0 0; 
     0 0 1 0 0 0 0 0; 
     1 0 0 0 0 0 0 0;
     0 1 0 0 0 0 0 0]
# N = 10
# A = [0 1 1 1 1 1 1 1 1 1;
#      1 0 0 0 0 0 0 0 0 0; 
#      1 0 0 0 0 0 0 0 0 0; 
#      1 0 0 0 0 0 0 0 0 0; 
#      1 0 0 0 0 0 0 0 0 0; 
#      1 0 0 0 0 0 0 0 0 0; 
#      1 0 0 0 0 0 0 0 0 0; 
#      1 0 0 0 0 0 0 0 0 0; 
#      1 0 0 0 0 0 0 0 0 0; 
#      1 0 0 0 0 0 0 0 0 0]

g = IndexedBiDiGraph(A)

# N = 20
# c = 3
# gg = erdos_renyi(N, c/N; seed)
# g = IndexedBiDiGraph(gg)
# A = zeros(Int,N,N)
# for i in CartesianIndices(A)
#      if !iszero(g.A[i])
#          A[i] = 1
#      end
# end

β = 1.0
J = 1
h = 0.0
m⁰ = 1.0
K = 100

svd_trunc=TruncBond(4)

A

8×8 Matrix{Int64}:
 0  1  1  1  0  0  1  0
 1  0  0  0  1  0  0  1
 1  0  0  0  0  1  0  0
 1  0  0  0  0  0  0  0
 0  1  0  0  0  0  0  0
 0  0  1  0  0  0  0  0
 1  0  0  0  0  0  0  0
 0  1  0  0  0  0  0  0

In [321]:
w = [fill(IntegerGlauberFactor([J*A[j,i] for j in inneighbors(g,i)], h, β), T+1) for i in vertices(g)]
ϕᵢ = [t == 0 ? [(1-m⁰)/2, (1+m⁰)/2] : ones(2) for t in 0:T]
# ϕᵢ = [ones(2) for t in 0:T]
bp = mpbp(g, w, fill(2,N), T; ϕ=fill(ϕᵢ,N))

for μ in bp.μ
    for a in μ
        a.=rand.(rng)
        # a.=1.0
    end
end
# bp_fourier = deepcopy(bp)
iterate!(bp, maxiter=2; svd_trunc)

msg = deepcopy(bp.μ)
# jldsave("messages.jld2"; msg)

bp_fourier = convert_msg_beliefs(ComplexF64, bp)
# iterate_fourier!(bp_fourier,K, maxiter=1, σ=1/100; svd_trunc)
;

Running MPBP: iter 2    Time: 0:00:00

In [322]:
# for (k,ed) in enumerate(edges(bp.g))
#     display(ed)
#     display(bp.μ[idx(ed)][3])
# end

In [323]:
iterate!(bp,maxiter=1; svd_trunc)

(1, CB_BP{ProgressUnknown, MatrixProductBP.var"#162#165"}(ProgressUnknown(false, false, 0, ProgressMeter.ProgressCore(:green, "Running MPBP: iter", 0.1, true, 0, VSCodeServer.IJuliaCore.IJuliaStdio{Base.PipeEndpoint, typeof(VSCodeServer.io_send_callback)}(IOContext(Base.PipeEndpoint(RawFD(25) open, 0 bytes waiting)), VSCodeServer.io_send_callback), false, 1, 1, ReentrantLock(nothing, 0x00000000, 0x00, Base.GenericCondition{Base.Threads.SpinLock}(Base.IntrusiveLinkedList{Task}(nothing, nothing), Base.Threads.SpinLock(0)), (132991630195264, 132991630197056, 132988893233200)), 0, 1, false, false, 1.737732011215156e9, 1.737732011215156e9, 1.737732011215156e9)), [[[2.0, 1.9998330208911554, 1.6254524515156616, 1.9578384244591172, 1.5710399696279016, 1.8937465353992748, 1.474298991815529, 1.8739023194200033, 1.4476364811673537, 1.9142230918470766, 1.4564445395976815], [2.0, 1.9975876955112795, 1.972805274858783, 1.9383417927605053, 1.9514182200857153, 1.889285623536548, 1.9338104786250179, 1.

In [324]:
iterate_fourier!(bp_fourier,K, maxiter=1, σ=1/100; svd_trunc)
;

In [325]:
import MatrixProductBP: marginalize
function marginalize(A::MPEM2{F}, var) where F
    if var==1
        return MPEM1{F}([@tullio b[m, n, xi] := a[m, n, xi, xj] for a in A]; z = A.z)
    elseif var==2
        return MPEM1{F}([@tullio b[m, n, xj] := a[m, n, xi, xj] for a in A]; z = A.z)
    end
end;

In [326]:
er_b = []
ptr = []
for i in vertices(bp.g)
    margs = marginals(bp.b[i])
    margs_fourier = marginals(bp_fourier.b[i])
    for t in 1:T+1
        marg = margs[t]
        marg_fourier = margs_fourier[t]
        ind = argmax(abs.(marg))
        push!(er_b, abs(marg_fourier[ind]-marg[ind]))
        push!(ptr, (i, t))
    end
end

display(maximum(er_b))
display(ptr[argmax(er_b)])

0.005004641418676403

(3, 7)

In [327]:
er_μ = []
ptr = []
for e in edges(bp.g)
    μ_marg = marginalize(bp.μ[idx(e)],1)
    margs = marginals(μ_marg)
    μ_fourier_marg = marginalize(bp_fourier.μ[idx(e)],1)
    margs_fourier = marginals(μ_fourier_marg)
    for t in 1:T+1
        marg = margs[t]
        marg_fourier = margs_fourier[t]
        push!(er_μ, abs(marg_fourier[1]-marg[1]))
        push!(ptr, (e, t))
    end
end

display(maximum(er_μ))
display(ptr[argmax(er_μ)])

0.004508653827397208

(Indexed Edge 2 => 8 with index 7, 8)

In [328]:
er_μ = []
ptr = []
for e in edges(bp.g)
    μ = bp.μ[idx(e)]
    μ_fourier = bp_fourier.μ[idx(e)]
    push!(er_μ, norm2m(μ, μ_fourier)/(norm(μ)*norm(μ_fourier)))
    push!(ptr, (e))
end

display(maximum(abs, er_μ))
display(ptr[argmax(abs.(er_μ))])

0.0007151611962720319

Indexed Edge 2 => 8 with index 7

In [329]:
bp_fourier.μ[1][3]

4×4×2×2 Array{ComplexF64, 4}:
[:, :, 1, 1] =
  -0.0105688-1.92632e-17im  …   -0.344549-0.000487437im
 -0.00989288-0.00148495im       -0.319254-0.0483827im
  2.21328e-5+1.3833e-6im       -0.0495725-0.00316869im
 7.65976e-19+2.10136e-5im      6.59103e-5-0.0465893im

[:, :, 2, 1] =
    -0.41176-3.84312e-16im  …    0.0183599+2.59739e-5im
  0.00175009+0.000262693im       0.0021116+0.000320012im
 0.000853871+5.3367e-5im        -0.0118659-0.00075847im
 2.82383e-20-3.32971e-6im      -5.03324e-7+0.000355779im

[:, :, 1, 2] =
   -0.0122143-2.43149e-17im  …    0.296929+0.000420068im
   -0.0114335-0.00171619im        0.275093+0.0416902im
  -1.43356e-5-8.95973e-7im      -0.0353323-0.00225845im
 -4.04567e-19-1.3606e-5im       4.69779e-5-0.0332067im

[:, :, 2, 2] =
    -0.475815-4.96861e-16im  …   -0.0158378-2.24059e-5im
   0.00202277+0.000303622im     -0.00176282-0.000267155im
 -0.000566144-3.5384e-5im         -0.012543-0.00080175im
 -3.34709e-21+2.20947e-6im      -3.79111e-7+0.000267978im

In [330]:
node = 2
time = 3

ed = inedges(bp.g, node) |> first
@show ed
μ = bp.μ[idx(ed)]
μ_marg = marginalize(μ,1)
marg = marginals(μ_marg)[time]

ed_fourier = inedges(bp_fourier.g, node) |> first
@show ed_fourier
μ_fourier = bp_fourier.μ[idx(ed_fourier)]
μ_fourier_marg = marginalize(μ_fourier,1)
marg_fourier = marginals(μ_fourier_marg)[time]

@show marg
@show marg_fourier;

ed = Indexed Edge 1 => 2 with index 1
ed_fourier = Indexed Edge 1 => 2 with index 1
marg = [0.07451493927100153, 0.9254850607289985]
marg_fourier = ComplexF64[0.07558333135216694 + 4.9973227521452986e-17im, 0.9244166686478331 - 4.9973227521452906e-17im]


In [331]:
node = 1
time = 3

edd = outedges(bp.g, node)
for ed in edd
    display(ed)
    μ = bp.μ[idx(ed)]
    μ_marg = marginalize(μ,1)
    marg = marginals(μ_marg)[time]

    μ_fourier = bp_fourier.μ[idx(ed)]
    μ_fourier_marg = marginalize(μ_fourier,1)
    marg_fourier = marginals(μ_fourier_marg)[time]

    display(marg)
    display(marg_fourier)
end

Indexed Edge 1 => 2 with index 1

2-element Vector{Float64}:
 0.07451493927100153
 0.9254850607289985

2-element Vector{ComplexF64}:
 0.07558333135216694 + 4.9973227521452986e-17im
  0.9244166686478331 - 4.9973227521452906e-17im

Indexed Edge 1 => 3 with index 2

2-element Vector{Float64}:
 0.06136183456388506
 0.9386381654361149

...

2-element Vector{Float64}:
 0.039076283535672364
 0.9609237164643276

2-element Vector{ComplexF64}:
 0.04026274516894552 + 7.546382934540041e-17im
  0.9597372548310547 - 7.546382934540032e-17im

Indexed Edge 1 => 7 with index 4

2-element Vector{Float64}:
 0.039076283535672045
 0.960923716464328

2-element Vector{ComplexF64}:
 0.04026274516894509 + 1.0069737541124327e-16im
   0.959737254831055 - 1.0069737541124325e-16im

In [332]:
for ed in edges(bp.g)
    display(ed)

    μ = bp.μ[idx(ed)]
    μ_marg = marginalize(μ,1)
    μ_fourier = bp_fourier.μ[idx(ed)]
    μ_fourier_marg = marginalize(μ_fourier,1)

    for time in 1:T+1
        marg = marginals(μ_marg)[time]
        marg_fourier = marginals(μ_fourier_marg)[time]
        display("time = $time")
        display(marg)
        display(marg_fourier)
    end
end

Indexed Edge 1 => 2 with index 1

"time = 1"

2-element Vector{Float64}:
 0.0
 1.0

2-element Vector{ComplexF64}:
 -4.0832238759116964e-18 - 1.2276614757613498e-18im
                     1.0 + 1.2276614757613612e-18im

"time = 2"

...

2-element Vector{Float64}:
 0.5
 0.5

2-element Vector{ComplexF64}:
                 0.5 - 4.960192837372201e-18im
 0.49999999999999994 + 4.960192837372201e-18im

"time = 11"

2-element Vector{Float64}:
 0.5000000000000001
 0.5

2-element Vector{ComplexF64}:
 0.5000000000000001 + 4.1645958785256594e-18im
 0.4999999999999999 - 4.164595878525661e-18im